In [1]:
import requests
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [2]:
baseurl = "https://api.wikiwho.net/en/api/v1.0.0-beta/"
content = "Mama's_Family"
def _url(url, path):
    return url + path
revisions_url = _url( baseurl,"rev_ids/")
mama_revisions_url = _url(revisions_url, content+"/")
params = {"editor": "true", "timestamp": "true"}
response = requests.get(mama_revisions_url, params= params)
revisons = pd.DataFrame(response.json()["revisions"])

In [3]:
revisons.head()

,editor,id,timestamp
0,0|172.194.5.181,3317807,2004-04-24T07:40:27Z
1,0|172.194.5.181,3317866,2004-04-24T07:49:11Z
2,41723,3317921,2004-04-24T07:58:03Z
3,44476,3317993,2004-04-24T08:09:12Z
4,41723,3493274,2004-04-24T08:21:07Z


In [4]:
revisons["id"].describe()

count    2.498000e+03
mean     3.350559e+08
std      2.327045e+08
min      3.317807e+06
25%      1.352418e+08
50%      2.149066e+08
75%      5.454177e+08
max      8.181112e+08
Name: id, dtype: float64

In [5]:
filename = content + ".h5"
with pd.HDFStore( filename, 'r') as store1:
    df_3317807 = store1.get("3317807")
    df_3317866 = store1.get("3317866")
    print(df_3317807.head(2))
    print("\n")   
    print(df_3317866.head(2))

   in  o_rev_id        out str  token_id  inserted  deleted
0  []   3317807  [3317921]  an         0      True    False
1  []   3317807  [3317921]  80         1      True    False


   in  o_rev_id        out str  token_id  inserted  deleted
0  []   3317807  [3317921]  an         0     False     True
1  []   3317807  [3317921]  80         1     False     True


In [6]:
inserted_array = np.array(df_3317866["inserted"])
inserted_pos = np.argwhere(inserted_array == True)
total_tokens = np.array(df_3317866 ["token_id"])
total_tokens = total_tokens.reshape(total_tokens.shape[0],1)
inserted_tokens = total_tokens[inserted_pos]
inserted_tokens[0]

array([[108]])

### code to find epsillon neighbourhood

In [7]:
pos = inserted_pos[0][0]
epsilon = 5
context =  np.concatenate((total_tokens[pos - epsilon: pos], total_tokens[pos+1: pos +epsilon]), axis=0)
# timing to select concatinate rather than vstack
# %timeit np.vstack((total_tokens[pos - epsilon: pos +epsilon], context))
# %timeit np.concatenate((total_tokens[pos - epsilon: pos +epsilon], context), axis=0)

for pos in np.nditer(inserted_pos[1:]):
    context= np.concatenate((context, total_tokens[pos - epsilon: pos], total_tokens[pos+1: pos +epsilon]), axis=0)

In [12]:
%timeit np.unique(context)

17.5 µs ± 401 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
%timeit set(context.flatten())

52 µs ± 649 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [129]:
context.shape

(477, 1)

In [133]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise

In [132]:
db = DBSCAN(eps=5, min_samples=1, p=1).fit(inserted_pos)

In [161]:
a = np.array([[1,2,3],[2,3,4]])
pairwise.pairwise_distances(a, metric = "jaccard")

array([[ 0.,  1.],
       [ 1.,  0.]])

In [225]:
unique_clusters = set(db.labels_) - {-1}

{0, 1}

In [10]:
ls

mama.h5                           wikiwho.ipynb
mama_700034288.h5                 wikiwho_context_clustering.ipynb
test.h5                           wikiwho_neighbour.ipynb
